In [6]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [5]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [3]:
from langchain_community.embeddings import OllamaEmbeddings
ollama_embedding = OllamaEmbeddings(model='llama2')

In [7]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=ollama_embedding)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is a process of breaking down a complex task into smaller, more manageable sub-tasks. It involves identifying the key components of the task and organizing them into a hierarchical structure, with each sub-task having a specific function or role in the overall process. By decomposing a task into smaller parts, it becomes easier to understand, plan, and execute the task more efficiently.\n\nIn the context of question-answering tasks, task decomposition can involve breaking down the task of finding relevant information into smaller sub-tasks, such as searching for relevant keywords, identifying key concepts, and organizing the found information into a logical structure. By decomposing the task in this way, it becomes easier to tackle each sub-task separately, using appropriate techniques and tools to complete each part of the task.\n\nIn the resources provided, task decomposition is mentioned in the context of vector similarity search. In this context, task decomposit